In [1]:
import pandas as pd
import numpy as np

import glob

import os
import numpy as np

import dask.dataframe as dd

In [2]:
# checking haiyao data

In [3]:
[x for x in [os.path.split(x)[1] for x in glob.iglob("/home/ted/data/final_feature/*")] if "djia_stock_v1" in x]

['djia_stock_v1_n10']

In [4]:
# !ls ../DeepTrader/data/DJIA

In [5]:
location = "/home/ted/data"
version = "djia_stock_v1"
n_prediction = 10

market_data_path = "/home/ted/data/raw/djia/data.pkl"

output_token_folder = "DJIA_v1"

output_version = f"{version}_n{n_prediction}"
final_feature_file = f"{location}/final_feature/{output_version}/data.pkl"

data_out = f"../DeepTrader/data/{output_token_folder}/"

In [6]:
final_feature_file

'/home/ted/data/final_feature/djia_stock_v1_n10/data.pkl'

# building data

In [7]:
df_tokens = pd.read_pickle(final_feature_file)

In [8]:
df_tokens

,,00_price_rising_rate0001,00_price_rising_rate0003,00_price_rising_rate0006,00_price_rising_rate0009,00_price_rising_rate0030,00_price_rising_rate0060,00_price_rising_rate0090,00_price_rising_rate0120,00_target,01_vol_7,...,01_vol_60,01_vol_90,01_vol_180,01_vol_360,02_trade_vol_1,02_trade_vol_3,02_trade_vol_7,02_trade_vol_30,02_trade_vol_60,02_trade_vol_90
date,stock_name,,,,,,,,,,,,,,,,,,,,,
1981-08-19,INTC,1.347883,1.055432,-0.783576,-1.414478,-0.698753,-2.112928,-1.032831,-0.970862,0.024389,0.003596,...,0.016225,0.016393,0.017535,0.032499,3484800.0,13670400.0,39350400.0,204796800.0,541603200.0,870115200.0
1981-08-20,INTC,1.991735,1.439910,1.746719,0.200678,-0.552163,-2.103513,-1.326483,-0.981572,-0.039683,0.003596,...,0.016077,0.016612,0.017475,0.032493,5577600.0,13920000.0,36988800.0,200044800.0,532262400.0,867436800.0
1981-08-21,INTC,-1.885045,0.456008,-0.403747,-0.736675,-1.184052,-2.147272,-1.537559,-1.093883,-0.016528,0.003025,...,0.015641,0.016884,0.017453,0.032511,8476800.0,17539200.0,41424000.0,201283200.0,526012800.0,867244800.0
1981-08-24,INTC,0.459583,0.045504,0.594965,-0.612037,-1.309778,-1.998272,-1.573805,-0.952140,-0.016805,0.003733,...,0.015060,0.017183,0.017383,0.032555,6441600.0,20496000.0,39782400.0,204278400.0,505363200.0,864796800.0
1981-08-25,INTC,-0.916318,-1.366914,-0.102815,0.106331,-1.319819,-1.772798,-1.708609,-0.965793,-0.025643,0.004159,...,0.014610,0.017508,0.017559,0.032611,13132800.0,28051200.0,47299200.0,207734400.0,507340800.0,840585600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,CRM,2.208215,0.090475,-0.525526,-0.990393,0.418297,1.150252,0.100756,-0.730036,-0.033887,5.870755,...,9.407252,10.298476,29.237392,39.151981,11438800.0,21698500.0,35853400.0,138261100.0,401663200.0,629483500.0
2022-08-25,CRM,-3.588170,-1.259835,-2.030746,-2.529826,-0.023651,0.917922,-0.289694,-1.100915,-0.049911,6.118278,...,9.278567,10.191764,28.953219,39.243692,24025600.0,40541200.0,56043100.0,156190600.0,407574200.0,648609600.0
2022-08-26,CRM,-1.231536,-2.039818,-2.404930,-2.244826,-0.495029,-0.306524,-0.581907,-1.158348,-0.030382,7.675522,...,9.142480,10.167686,28.602810,39.369134,11074700.0,46539100.0,63936500.0,161227600.0,381611500.0,653563900.0


In [9]:
tokens = sorted(df_tokens.index.get_level_values(1).unique().tolist())

In [10]:
len(tokens)

30

In [11]:
df_tokens.columns.tolist()

['00_price_rising_rate0001',
 '00_price_rising_rate0003',
 '00_price_rising_rate0006',
 '00_price_rising_rate0009',
 '00_price_rising_rate0030',
 '00_price_rising_rate0060',
 '00_price_rising_rate0090',
 '00_price_rising_rate0120',
 '00_target',
 '01_vol_7',
 '01_vol_30',
 '01_vol_60',
 '01_vol_90',
 '01_vol_180',
 '01_vol_360',
 '02_trade_vol_1',
 '02_trade_vol_3',
 '02_trade_vol_7',
 '02_trade_vol_30',
 '02_trade_vol_60',
 '02_trade_vol_90']

# determind by stock still exsit over years

In [12]:
df_first = df_tokens.reset_index().groupby("stock_name")["date"].first()
df_last = df_tokens.reset_index().groupby("stock_name")["date"].last()

first_date_of_first = df_first.value_counts().sort_index().cumsum().pipe(lambda x: x[x >= 27]).index[0]
tokens_trading1 = df_first.pipe(lambda x: x[x <= first_date_of_first]).index.tolist()
tokens_trading2 = df_last.pipe(lambda x: x[x == x.max()]).index.tolist()
tokens_trading = set(tokens_trading1) & set(tokens_trading2)

df_tokens_keep_pre = df_tokens.pipe(lambda x: x[x.index.get_level_values(1).isin(tokens_trading) \
                                            & (x.index.get_level_values(0) >= first_date_of_first)])

In [13]:
df_tokens_keep_pre

,,00_price_rising_rate0001,00_price_rising_rate0003,00_price_rising_rate0006,00_price_rising_rate0009,00_price_rising_rate0030,00_price_rising_rate0060,00_price_rising_rate0090,00_price_rising_rate0120,00_target,01_vol_7,...,01_vol_60,01_vol_90,01_vol_180,01_vol_360,02_trade_vol_1,02_trade_vol_3,02_trade_vol_7,02_trade_vol_30,02_trade_vol_60,02_trade_vol_90
date,stock_name,,,,,,,,,,,,,,,,,,,,,
2000-10-04,INTC,1.394126,0.139035,-0.173237,-2.020986,-2.364051,-1.844541,-1.379375,-1.152715,-0.023809,1.151680,...,5.872708,4.896719,4.587292,8.584094,63726000.0,211718200.0,544489500.0,1.876167e+09,2.983544e+09,4.144482e+09
2000-10-05,INTC,-1.010794,0.278922,-0.514371,-0.927858,-2.315587,-1.908170,-1.536294,-1.088567,-0.025915,0.976333,...,6.113580,5.101856,4.649630,8.568588,52072400.0,197004900.0,483880300.0,1.892062e+09,3.005792e+09,4.154787e+09
2000-10-06,INTC,-0.281964,-0.047935,-0.524621,-0.686013,-2.346360,-1.957997,-1.570582,-1.271767,-0.021909,1.018147,...,6.347655,5.325326,4.708550,8.556474,69734600.0,185533000.0,473249800.0,1.938444e+09,3.030351e+09,4.170432e+09
2000-10-09,INTC,-0.802408,-1.265072,-0.640160,-0.518221,-2.304824,-2.006670,-1.695167,-1.358170,-0.038400,0.990455,...,6.569908,5.550420,4.780934,8.540747,56544700.0,178351700.0,463055500.0,1.974145e+09,3.049424e+09,4.180998e+09
2000-10-10,INTC,-1.258316,-1.121331,-0.623429,-0.819751,-2.274132,-1.990535,-1.716858,-1.290715,-0.058237,0.847324,...,6.784569,5.788608,4.868387,8.528221,68882800.0,195162100.0,458952700.0,2.000267e+09,3.071726e+09,4.207224e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,JNJ,-0.380038,0.431893,1.027782,0.340065,-1.332492,-0.602632,-0.118347,0.323349,0.009361,1.062141,...,4.335280,4.531044,5.795851,7.196711,5684300.0,16713200.0,48014000.0,1.998705e+08,4.152658e+08,6.456524e+08
2022-08-25,JNJ,-0.075633,-0.540933,0.877209,1.136761,-1.310097,-0.572478,-0.002876,0.234897,-0.017112,0.920492,...,4.282097,4.574478,5.742252,7.170502,5678700.0,16929300.0,46239100.0,2.001896e+08,4.045531e+08,6.450124e+08
2022-08-26,JNJ,0.815753,0.420058,1.424370,1.101853,-1.188091,-0.387535,-0.022651,-0.000004,-0.007731,1.250736,...,4.331442,4.684123,5.707374,7.143940,5667100.0,17030100.0,44499600.0,2.001656e+08,4.050448e+08,6.362987e+08


## reserved using for industry classification

In [14]:
cut_date_index = df_tokens_keep_pre.head(90).index.get_level_values(0)

In [15]:
df_tokens_keep = df_tokens_keep_pre[~df_tokens_keep_pre.index.get_level_values(0).isin(cut_date_index)]

In [16]:
df_tokens_keep_res = df_tokens_keep_pre[df_tokens_keep_pre.index.get_level_values(0).isin(cut_date_index)]

In [17]:
df_tokens_keep.shape

(146367, 21)

In [18]:
df_tokens_keep_res.shape

(2430, 21)

## ror build

In [19]:
df_ror = df_tokens_keep[["00_target"]]
targets = [x for x in df_tokens_keep.columns.tolist() if "target" in x]
df_stock_data = df_tokens_keep.drop(targets, 1)
df_ror = df_ror.unstack("date")
df_ror.sort_index(inplace=True)

/tmp/ipykernel_22880/3470666270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_stock_data = df_tokens_keep.drop(targets, 1)


In [20]:
df_ror.shape

(27, 5421)

In [21]:
df_ror.head()

00_target                                                         \
date       2001-02-13 2001-02-14 2001-02-15 2001-02-16 2001-02-20 2001-02-21   
stock_name                                                                     
AAPL         0.019606   0.028847  -0.052959  -0.036184   0.030717  -0.003311   
AMGN        -0.004322   0.000868  -0.011275  -0.039474   0.031963  -0.007965   
AXP         -0.002992   0.004717   0.007681  -0.026259  -0.043715   0.012054   
BA          -0.002141  -0.007262  -0.002660   0.031005  -0.026678   0.017941   
CAT         -0.007860   0.027959  -0.010200  -0.042822  -0.020095  -0.006592   

                                                        ...             \
date       2001-02-22 2001-02-23 2001-02-26 2001-02-27  ... 2022-08-17   
stock_name                                              ...              
AAPL         0.000000   0.036542  -0.006410  -0.058064  ...  -0.002292   
AMGN         0.005353   0.041703   0.007666  -0.025359  ...  -0.003512   
AXP          0.008988   0.006681   0.005310  -0.034330  ...   0.004576   
BA           0.005385   0.018665   0.014978  -0.023547  ...   0.008912   
CAT         -0.014746   0.038414   0.020658  -0.020946  ...   0.007248   

                                                                              \
date       2022-08-18 2022-08-19 2022-08-22 2022-08-23 2022-08-24 2022-08-25   
stock_name                                                                     
AAPL        -0.015102  -0.023029  -0.002029   0.001794   0.014923  -0.037699   
AMGN         0.004646  -0.015786  -0.009842   0.000082   0.004867  -0.020474   
AXP         -0.014637  -0.027983  -0.002346   0.011441   0.019481  -0.030327   
BA          -0.034205  -0.024184   0.006856   0.022053   0.035330  -0.028634   
CAT         -0.008867  -0.019632   0.028421  -0.015466   0.024310  -0.034996   

                                             
date       2022-08-26 2022-08-29 2022-08-30  
stock_name                                   
AAPL        -0.013690  -0.015305  -0.010635  
AMGN        -0.005277  -0.001086   0.004935  
AXP         -0.017609   0.000777  -0.017199  
BA           0.005409  -0.019405  -0.012083  
CAT         -0.000729  -0.025237  -0.011929  

[5 rows x 5421 columns]

In [22]:
dd_ror = df_ror.values

# build stock data

In [23]:
number_of_days = df_ror.shape[1]
number_of_tokens = df_ror.shape[0]
number_of_features = df_stock_data.shape[1]

dd_stock_data = df_stock_data.sort_index().values.reshape(number_of_days, number_of_tokens, number_of_features)

In [24]:
dd_stock_data.shape

(5421, 27, 20)

In [25]:
dd_stock_data = dd_stock_data.swapaxes(0, 1)

In [26]:
df_stock_data.sort_index()

00_price_rising_rate0001  00_price_rising_rate0003  \
date       stock_name                                                       
2001-02-13 AAPL                       -0.910441                 -2.215777   
           AMGN                       -0.757866                  1.107081   
           AXP                        -0.867538                 -0.133408   
           BA                          1.703215                  1.689597   
           CAT                         0.550603                  0.118372   
...                                         ...                       ...   
2022-08-30 TRV                        -0.116447                  0.659365   
           UNH                        -0.313617                  0.567480   
           VZ                         -0.980542                  0.899022   
           WBA                        -1.380521                  0.115423   
           WMT                         0.934366                  1.028992   

                       00_price_rising_rate0006  00_price_rising_rate0009  \
date       stock_name                                                       
2001-02-13 AAPL                       -0.837786                 -1.560444   
           AMGN                        1.345205                  0.633257   
           AXP                        -0.465537                  0.096619   
           BA                          1.203006                  0.825505   
           CAT                        -0.033205                 -0.130515   
...                                         ...                       ...   
2022-08-30 TRV                         0.184882                  0.132318   
           UNH                        -0.047467                  0.677033   
           VZ                          0.054485                  0.087588   
           WBA                        -0.679151                 -1.765619   
           WMT                         0.848334                  0.406082   

                       00_price_rising_rate0030  00_price_rising_rate0060  \
date       stock_name                                                       
2001-02-13 AAPL                        1.909262                 -0.291031   
           AMGN                        0.815464                  0.377242   
           AXP                        -1.150919                 -1.194502   
           BA                         -0.656495                 -0.315783   
           CAT                        -0.666799                  1.068521   
...                                         ...                       ...   
2022-08-30 TRV                         0.555012                 -0.063329   
           UNH                        -0.204212                  1.388748   
           VZ                         -2.199773                 -1.078002   
           WBA                        -1.046705                 -1.315161   
           WMT                         0.478170                  1.189995   

                       00_price_rising_rate0090  00_price_rising_rate0120  \
date       stock_name                                                       
2001-02-13 AAPL                       -1.159765                 -2.439107   
           AMGN                        0.498711                  0.195874   
           AXP                        -1.371287                 -0.727822   
           BA                         -0.157424                  0.937791   
           CAT                         1.202846                  0.657891   
...                                         ...                       ...   
2022-08-30 TRV                         0.433988                  0.183706   
           UNH                         0.964432                  1.315603   
           VZ                         -1.367848                 -1.064434   
           WBA                        -1.529955                 -1.723690   
           WMT                        -0.735496                  0.204988   

               

In [27]:
dd_stock_data.shape

(27, 5421, 20)

In [28]:
pd.DataFrame(dd_stock_data[:, 0, :]).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.910441,-2.215777,-0.837786,-1.560444,1.909262,-0.291031,-1.159765,-2.439107,0.011823,0.031267,0.036292,0.037875,0.264282,0.247966,237070400.0,1.101274e+09,2.848115e+09,1.735461e+10,2.988028e+10,5.067881e+10
1,-0.757866,1.107081,1.345205,0.633257,0.815464,0.377242,0.498711,0.195874,1.439851,3.727266,2.843481,2.984629,3.241785,7.625183,8579600.0,3.011550e+07,5.780450e+07,3.211453e+08,5.234026e+08,8.033535e+08
2,-0.867538,-0.133408,-0.465537,0.096619,-1.150919,-1.194502,-1.371287,-0.727822,0.480611,1.694770,2.393199,2.734302,2.617001,3.036886,5539586.0,1.342454e+07,3.721056e+07,2.424337e+08,3.875488e+08,5.304643e+08
3,1.703215,1.689597,1.203006,0.825505,-0.656495,-0.315783,-0.157424,0.937791,0.903066,1.466969,2.898707,2.594754,6.197254,6.899346,4984900.0,9.877700e+06,1.941780e+07,1.082816e+08,2.243571e+08,3.419955e+08
4,0.550603,0.118372,-0.033205,-0.130515,-0.666799,1.068521,1.202846,0.657891,0.119218,0.482558,0.831336,1.468931,1.220750,1.848687,2898400.0,9.200800e+06,1.704980e+07,1.224466e+08,2.282198e+08,3.241306e+08


## build market data

In [29]:
df_market_data = pd.read_pickle(market_data_path)
df_market_data = df_market_data.dropna()
df_market_data = df_market_data.loc[np.unique(df_stock_data.index.get_level_values(0).date)]

In [30]:
df_market_data

,Close,Volume
2001-02-13,0.379654,-0.783114
2001-02-14,-0.732006,-0.965283
2001-02-15,0.078429,0.197186
2001-02-16,-1.409025,-0.392256
2001-02-20,-1.919898,0.085682
...,...,...
2022-08-24,-0.833739,-1.414705
2022-08-25,-0.283965,1.107846
2022-08-26,-2.006760,2.312069
2022-08-29,-1.963447,0.859624


In [31]:
dd_market_data = df_market_data.values

##  build classifiction

In [41]:
with open("../DeepTrader/data/DJIA/industry_classification.npy", "rb") as file:
    indu_array =  np.load(file)

In [33]:
# reading all classification

file_paths = list(glob.iglob("../DeepTrader/data/DJIA/djia_sectors_stocks/*.csv"))
df_indu = dd.read_csv(file_paths).compute()

keep_stocks = df_ror.index
df_indu_kep = df_indu[df_indu.Symbol.isin(keep_stocks)]
df_sector_pre = df_indu_kep[["Symbol", "Sector"]]

s = df_sector_pre.groupby("Sector").count().apply(lambda x: 1/ x)
s.columns = ["val"]

df_sector = df_sector_pre.merge(s, on="Sector", how="left")
s_sector = df_sector.merge(df_sector, on="Sector", how="outer").set_index(["Symbol_x", "Symbol_y"])["val_x"]
df_indu2 = s_sector.unstack().fillna(0)

In [40]:
df_indu2

Symbol_y,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
Symbol_x,,,,,,,,,,,,,,,,,,,,,
AAPL,0.25,0.00,0.00,0.00,0.00,0.0,0.0,0.000000,0.00,0.000000,...,0.00,0.00,0.25,0.000000,0.000000,0.00,0.00,0.0,0.0,0.000000
AMGN,0.00,0.25,0.00,0.00,0.00,0.0,0.0,0.000000,0.00,0.000000,...,0.00,0.25,0.00,0.000000,0.000000,0.00,0.25,0.0,0.0,0.000000
AXP,0.00,0.00,0.25,0.00,0.00,0.0,0.0,0.000000,0.25,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.25,0.00,0.0,0.0,0.000000
BA,0.00,0.00,0.00,0.25,0.25,0.0,0.0,0.000000,0.00,0.000000,...,0.25,0.00,0.00,0.000000,0.000000,0.00,0.00,0.0,0.0,0.000000
CAT,0.00,0.00,0.00,0.25,0.25,0.0,0.0,0.000000,0.00,0.000000,...,0.25,0.00,0.00,0.000000,0.000000,0.00,0.00,0.0,0.0,0.000000
CSCO,0.00,0.00,0.00,0.00,0.00,0.5,0.0,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.5,0.0,0.000000
CVX,0.00,0.00,0.00,0.00,0.00,0.0,1.0,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.0,0.0,0.000000
DIS,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.166667,0.00,0.166667,...,0.00,0.00,0.00,0.166667,0.166667,0.00,0.00,0.0,0.0,0.166667
GS,0.00,0.00,0.25,0.00,0.00,0.0,0.0,0.000000,0.25,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.25,0.00,0.0,0.0,0.000000


In [34]:
dd_industry_classification = df_indu2.values

## output

In [35]:
try:
    os.mkdir(data_pattern)
except:
    pass

for x, n in zip([dd_industry_classification, dd_ror, dd_stock_data, dd_market_data], ["industry_classification", "ror", "stocks_data", "market_data"]):
    print(f"{n}:", x.shape)
    np.save(f"{data_out}/{n}.npy", x)

industry_classification: (27, 27)
ror: (27, 5421)
stocks_data: (27, 5421, 20)
market_data: (5421, 2)
